In [2]:
import time
import os
import pandas as pd
import numpy as np
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
data_path="../../data/"
output_path="../../outputs/"

In [3]:
positives=pd.read_csv(output_path+"sg_ie/positives_ready.csv")

In [4]:
positives=positives.assign(abstract_for_prompt=positives.abstract_for_prompt.fillna(""))
positives=positives.assign(acknowledgments_for_prompt=positives.acknowledgments_for_prompt.fillna(""))

positives=positives.assign(paper_text="Title: "+positives.title_clean+"\n"+positives.abstract_for_prompt+
                          positives.acknowledgments_for_prompt)

In [5]:
ack_preprompt="""Identify the organizations mentioned in the following paper. Respond with the organizations separated by commas. Answer "No organizations" if there aren't any organizations in the text:"""

ack_postprompt="""Which are the organizations mentioned?"""

In [6]:
positives=positives.assign(org_prompt_text=ack_preprompt+"\n"+positives.paper_text+"\n"+ack_postprompt)

## requests

In [8]:
positives=positives.loc[~positives.acknowledgments_clean.isna()].reset_index(drop=True)

In [ ]:
for i,d in positives.iterrows():
    input_prompt=d['org_prompt_text']
    completion = openai.Completion.create(engine="text-davinci-002", prompt=input_prompt,temperature=0,max_tokens=40)
    positives.loc[i,'GPT3_response']=completion.choices[0].text
    f = open("responses.txt", "a")
    f.write(d['org_prompt_text'])
    f.write(completion.choices[0].text)
    f.write("################################")
    f.write("\n")
    f.close()
    print(input_prompt)
    print(completion.choices[0].text)
    print("################################")

In [10]:
positives.to_csv(output_path+"GPT3_responses_org.csv",index=False)

In [16]:
positives=positives.assign(clean_response=positives.GPT3_response.replace("\n","",regex=True))

positives=positives.assign(clean_response=positives.clean_response.str.split(","))

orgs=positives.loc[:,['ID','clean_response']]

orgs=orgs.explode("clean_response")

orgs=orgs.rename(columns={'clean_response':'organization'})

orgs=orgs.assign(organization=orgs.organization.str.lstrip().str.rstrip())

In [36]:
orgs.to_csv(output_path+"organizations_GPT3.csv",index=False)

In [35]:
orgs.organization.value_counts()

No organizations                               770
DARPA                                           95
National Science Foundation                     68
NSF                                             67
National Institutes of Health                   43
                                              ... 
MRC Skills Development Fellowship programme      1
Kings College London                             1
MR/S00310X/1                                     1
UKRI Innovation Fellowship                       1
King Saud University                             1
Name: organization, Length: 2315, dtype: int64